In [ ]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import Autoformer, PatchTST
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess your dataset
Y_df = pd.read_csv('../data/average_weekly_rating_messenger.csv')

# Ensure the 'ds' column is datetime
Y_df['ds'] = pd.to_datetime(Y_df.iloc[:, 0])

# Ensure the 'y' column is numeric and handle any non-numeric issues
Y_df['y'] = pd.to_numeric(Y_df.iloc[:, 1], errors='coerce')
Y_df.dropna(subset=['y'], inplace=True)  # Drop rows where 'y' is NaN after conversion

# Add unique_id as a constant value
Y_df['unique_id'] = 'rating'

# Keep only necessary columns
Y_df = Y_df[['unique_id', 'ds', 'y']]

In [ ]:
print(Y_df.dtypes) 
print(Y_df.head())

In [ ]:
horizon = 48  # Forecasting 48 weeks (around 1 year)

# Define the Informer model
models = [
    Autoformer(
        h=horizon,                 
        input_size=48,               
        max_steps=100,              
        val_check_steps=100,         
    ),
    PatchTST(
        h=horizon,                  
        input_size=48,               
        max_steps=100,              
        val_check_steps=100,        
    ),
]

In [ ]:
nf = NeuralForecast(
    models=models,
    freq='W'  # Weekly frequency for your dataset
)

# Fit the model on the dataset (training the model)
nf.fit(df=Y_df)

# Predict future values using the trained model (horizon is defined above)
Y_hat_future = nf.predict().reset_index(drop=True)